## Download do Dataset

In [3]:
!pip install kagglehub


import kagglehub

# Download latest version
path = kagglehub.dataset_download("lucasmoraes001/orange-train-test")

print("Path to dataset files:", path)

100%|██████████| 5.69G/5.69G [00:41<00:00, 148MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/lucasmoraes001/orange-train-test/versions/1


## Importando dados

In [1]:
import os
import cv2 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision import models
from torchvision.io import decode_image
from torchvision.models import resnet50, ResNet50_Weights
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from multiprocessing import Pool
import os
import shutil
import mlflow
from torcheval.metrics import MulticlassF1Score, MulticlassPrecision, MulticlassRecall, MulticlassConfusionMatrix
import mlflow.pytorch
from mlflow import MlflowClient


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def cv2_imshow(img, show=True):
    if img is None:
        raise FileNotFoundError("Imagem não encontrada")

    #img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    if show:
        # Exibe a imagem diretamente com OpenCV
        cv2.imshow('Imagem lida com cv2', img)
        cv2.waitKey(0)  # Espera até pressionar qualquer tecla
        cv2.destroyAllWindows()  # Fecha a janela depois
    
    return img


In [ ]:
# Verificando tamanho das imagens

tamanho = cv2.imread(r'C:\Users\Lucas\Documents\GitHub\Computer_Vision_ML\computer_vision\data\processed\train\citrus canker\c (1).jpg')

cv2_imshow(tamanho)

## Tratando

In [2]:
# Funções 

def salvar_dados(tensores_imagens, classes, nome_arquivo):
    # Salva tanto os tensores de imagem quanto as classes em um único arquivo
    torch.save({
        'imagens': tensores_imagens,
        'classes': classes
    }, nome_arquivo)
    print(f'Dados salvos com sucesso em {nome_arquivo}')

def descobrir_classe(path):
    nome = os.path.basename(os.path.dirname(path)).lower()
    if 'canker' in nome:
        return 'citrus_canker'
    elif 'healthy' in nome:
        return 'healthy'
    elif 'melanose' in nome:
        return 'melanose'
    else:
        return 'unknown'

def copiar_arquivos(lista_paths, destino_base):
    for path in lista_paths:
        classe = descobrir_classe(path)
        destino_classe = os.path.join(destino_base, classe)
        os.makedirs(destino_classe, exist_ok=True)

        filename = os.path.basename(path)
        dest_path = os.path.join(destino_classe, filename)
        shutil.copy(path, dest_path)



In [11]:
path = r'C:\Users\Lucas\Documents\GitHub\Computer_Vision_ML\computer_vision\data\raw'

caminhos = os.listdir(path)
citrus_canker = [os.path.abspath(os.path.join(path, caminhos[1], file)) for file in os.listdir(os.path.join(path, caminhos[1]))]
healthy = [os.path.abspath(os.path.join(path, caminhos[2], file)) for file in os.listdir(os.path.join(path, caminhos[2]))]
melanose = [os.path.abspath(os.path.join(path, caminhos[3], file)) for file in os.listdir(os.path.join(path, caminhos[3]))]

In [ ]:
# Criando amostras
print('\nCriando amostras')
amostras_0 = citrus_canker[:100] + healthy[:100] + melanose[:100]
classe_amostra = []

classe_amostra = classe_treatment(amostras_0)

# Processando amostras com Torchvision
amostras_0 = img_processing(amostras_0, device)

# Aplicando torch.stack nas amostras também
amostras_0 = torch.stack(amostras_0)

print('\nAmostras criadas!')



Criando amostras


In [ ]:
import os
import shutil
 
def salvar_dados(tensores_imagens, classes, nome_arquivo):
    # Salva tanto os tensores de imagem quanto as classes em um único arquivo
    torch.save({
        'imagens': tensores_imagens,
        'classes': classes
    }, nome_arquivo)
    print(f'Dados salvos com sucesso em {nome_arquivo}')

def descobrir_classe(path):
    nome = os.path.basename(os.path.dirname(path)).lower()
    if 'canker' in nome:
        return 'citrus_canker'
    elif 'healthy' in nome:
        return 'healthy'
    elif 'melanose' in nome:
        return 'melanose'
    else:
        return 'unknown'

def copiar_arquivos(lista_paths, destino_base):
    for path in lista_paths:
        classe = descobrir_classe(path)
        destino_classe = os.path.join(destino_base, classe)
        os.makedirs(destino_classe, exist_ok=True)

        filename = os.path.basename(path)
        dest_path = os.path.join(destino_classe, filename)
        shutil.copy(path, dest_path)

path = 'data/path'

caminhos = os.listdir(path)
citrus_canker = [os.path.abspath(os.path.join(path, caminhos[1], file)) for file in os.listdir(os.path.join(path, caminhos[1]))]
healthy = [os.path.abspath(os.path.join(path, caminhos[2], file)) for file in os.listdir(os.path.join(path, caminhos[2]))]
melanose = [os.path.abspath(os.path.join(path, caminhos[3], file)) for file in os.listdir(os.path.join(path, caminhos[3]))]

print('Iniciando tratamentos dos dados e a separação da mostra...\n')

if len(citrus_canker) != len(healthy) or len(citrus_canker) != len(melanose):
    print('Classes desequilibradas:\n')
    print(f'Número de Citrus Canker: {len(citrus_canker)}')
    print(f'Número de Healthy: {len(healthy)}')
    print(f'Número de Melanose: {len(melanose)}')

    # Balanceando as classes
    min_len = min(len(citrus_canker), len(healthy), len(melanose))
    citrus_canker = citrus_canker[:min_len]
    healthy = healthy[:min_len]
    melanose = melanose[:min_len]

    print('\nClasses atualizadas:\n')
    print(f'Número de Citrus Canker: {len(citrus_canker)}')
    print(f'Número de Healthy: {len(healthy)}')
    print(f'Número de Melanose: {len(melanose)}')

    # Separando Treino, Teste e Avaliação
    train = citrus_canker[:2000] + healthy[:2000] + melanose[:2000]
    test = citrus_canker[2000:2300] + healthy[2000:2300] + melanose[2000:2300]
    eval = citrus_canker[2300:2600] + healthy[2300:2600] + melanose[2300:2600]

    # Define o diretório base onde vai salvar
    base_dir = r'C:\Users\Lucas\Documents\GitHub\Computer_Vision_ML\computer_vision\data\processed'

    # Define as pastas de destino
    dirs = {
        'train': os.path.join(base_dir, 'train'),
        'test': os.path.join(base_dir, 'test'),
        'eval': os.path.join(base_dir, 'eval')
    }

    # Cria as pastas principais se não existirem
    for pasta in dirs.values():
        os.makedirs(pasta, exist_ok=True)

    # Copiando os arquivos
    print('\nCopiando arquivos de treino...')
    copiar_arquivos(train, dirs['train'])

    print('Copiando arquivos de teste...')
    copiar_arquivos(test, dirs['test'])

    print('Copiando arquivos de avaliação...')
    copiar_arquivos(eval, dirs['eval'])

    print('\nArquivos copiados com sucesso!')

else:
    print('Classes estão equilibradas ok')




Iniciando tratamentos dos dados e a separação da mostra...

Classes desequilibradas:

Número de Citrus Canker: 3000
Número de Healthy: 3000
Número de Melanose: 2600

Classes atualizadas:

Número de Citrus Canker: 2600
Número de Healthy: 2600
Número de Melanose: 2600

Copiando arquivos de treino...
Copiando arquivos de teste...
Copiando arquivos de avaliação...

Arquivos copiados com sucesso!


### Criando Dataloaders para dados de treino

In [3]:


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    #v2.Grayscale(num_output_channels=1), Resnet espera 3 canais
    transforms.ToTensor()
])

dataset_test = ImageFolder(root=r'C:\Users\Lucas\Documents\GitHub\OrangeDetect\data\processed\test', transform=transform) # Automaticamente classifica como 0, 1 e 2 respectivamente a ordem da pasta
dataset_train = ImageFolder(root=r'C:\Users\Lucas\Documents\GitHub\OrangeDetect\data\processed\train', transform=transform)
dataset_eval = ImageFolder(root=r'C:\Users\Lucas\Documents\GitHub\OrangeDetect\data\processed\eval', transform=transform)



train_dataloader = DataLoader(dataset_train, batch_size=32, num_workers=4,shuffle=True)
test_dataloader = DataLoader(dataset_test, batch_size=32, num_workers=4,shuffle=True)
eval_dataloader = DataLoader(dataset_eval, batch_size=32, num_workers=4,shuffle=True)

        

In [41]:
dataset_train[0][0]

tensor([[[0.9294, 0.8706, 0.7647,  ..., 0.7451, 0.7451, 0.7333],
         [0.9333, 0.9216, 0.9059,  ..., 0.7412, 0.7373, 0.7294],
         [0.9333, 0.9373, 0.9255,  ..., 0.7412, 0.7373, 0.7294],
         ...,
         [0.4353, 0.4353, 0.4392,  ..., 0.6510, 0.6510, 0.6510],
         [0.4431, 0.4431, 0.4431,  ..., 0.6431, 0.6471, 0.6510],
         [0.4431, 0.4431, 0.4431,  ..., 0.6431, 0.6431, 0.6471]],

        [[0.9294, 0.8667, 0.7569,  ..., 0.7765, 0.7765, 0.7647],
         [0.9333, 0.9176, 0.8980,  ..., 0.7725, 0.7686, 0.7608],
         [0.9294, 0.9294, 0.9176,  ..., 0.7725, 0.7686, 0.7608],
         ...,
         [0.4784, 0.4784, 0.4824,  ..., 0.6941, 0.6941, 0.6941],
         [0.4863, 0.4863, 0.4863,  ..., 0.6863, 0.6902, 0.6941],
         [0.4863, 0.4863, 0.4863,  ..., 0.6863, 0.6863, 0.6902]],

        [[0.9686, 0.9137, 0.8078,  ..., 0.8510, 0.8510, 0.8392],
         [0.9725, 0.9647, 0.9490,  ..., 0.8471, 0.8431, 0.8353],
         [0.9725, 0.9765, 0.9686,  ..., 0.8471, 0.8431, 0.

In [43]:
to_pil = transforms.ToPILImage()
image2 = to_pil(dataset_train[0][0])
image2.show()

## Feature Extractor com Resnet50


### Entendendo Resnet

In [16]:
weights = ResNet50_Weights.DEFAULT

model = resnet50(weights=weights).to(device='cuda')  # não esquece de inicializar o modelresnet50


In [7]:
for img, label in train_dataloader:
    img = img.to(device='cuda')
    break

In [8]:
img.shape # O batch de 32 imagens 224x224 com 1 canal apenas

torch.Size([32, 3, 224, 224])

In [17]:
from torchvision.io import decode_image
from torchvision.models import resnet50, ResNet50_Weights


# Step 1: Initialize model with the best available weights
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img[0]).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

NameError: name 'img' is not defined

## Começando


### Testando 

In [4]:
weights = ResNet50_Weights.DEFAULT

model = resnet50(weights=weights).to(device)

for param in model.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)


criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized
optimizer_conv = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [27]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
from torchsummary import summary

model.to(device='cuda')
summary(model, input_size=(3, 224, 224), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [6]:
# Testando

# Usando o test_dataloader apenas para testar o código, pois este dataloader tem apenas 300 imagens.

model.train()
epochs = 3
loss_list = []

size = len(test_dataloader.dataset)

for epoch in range(epochs):
    print(f'------Época {epoch+1}------')
    for batch, (X, y) in enumerate(test_dataloader):
        X, y = X.to(device='cuda'), y.to(device='cuda')
    
        
        pred = model(X)
        loss = criterion(pred, y)

        loss.backward()
        optimizer_conv.step()
        optimizer_conv.zero_grad()
        loss_list.append(loss.item())

        if batch % 100 == 0:
            print(f'Loss Atual: {loss.item()}')

    print('Avaliando o modelo')
    size_eval = len(eval_dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X_test, y_test in eval_dataloader:
            X_test = X_test.to(device='cuda')
            y_test = y_test.to(device='cuda')
            pred = model(X_test)
            test_loss += criterion(pred, y_test).item()
            predicted = (pred > 0.5).float()
            predicted = predicted.argmax(dim=1)
            correct += (predicted == y_test).sum().item()
    accuracy = correct / size_eval
    print(f'Acurácia é {accuracy * 100:.2f}%')
    model.train()  # Volta pro modo de treino


------Época 1------
Loss Atual: 1.0754526853561401
Avaliando o modelo
Acurácia é 34.11%
------Época 2------
Loss Atual: 0.6599974036216736


KeyboardInterrupt: 

In [10]:
from matplotlib.pyplot import plot

plot(loss_list)

: 

In [9]:
model.eval()
test_loss, correct = 0, 0

y_pred = []
y = []

with torch.no_grad():
    for X_test, y_test in eval_dataloader:
        X_test = X_test.to(device='cuda')
        y_test = y_test.to(device='cuda')
        pred = model(X_test)
        test_loss += criterion(pred, y_test).item()
        predicted = (pred > 0.5).float()
        predicted = predicted.argmax(dim=1)
        y_pred.append(predicted)
        y.append(y_test)
        



KeyboardInterrupt: 

In [49]:
y_pred = torch.cat(y_pred, dim=0) 
y = torch.cat(y, dim=0) 

In [ ]:
from torcheval.metrics.functional import multiclass_f1_score, multiclass_precision, multiclass_recall
input = torch.tensor([0, 2, 1, 3])
target = torch.tensor([0, 1, 2, 3])
multiclass_f1_score(y_pred, y, num_classes=3)

tensor(0.3333, device='cuda:0')

### Experimento

In [2]:
# Importando modelo e seus pesos
weights = ResNet50_Weights.DEFAULT

model = resnet50(weights=weights)

# Congelando as camadas de convolução

for param in model.parameters():
    param.requires_grad = False

# Ativando apenas as camadas FC (camadas lineares)

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)


criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [13]:
model.__class__

torchvision.models.resnet.ResNet

In [8]:
# Accessing the model directly from the library dictionary
library['model']

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# Modelo para GPU

model.to(device)


# Iniciar um experimento no MLflow

mlflow.set_experiment("Orange Detect")

mlflow.pytorch.autolog()

# Iniciar o run no MLflow
with mlflow.start_run() as run:

    # Logar hiperparâmetros
    params = {
        'Loss Function' : criterion,
        'Optimizer' : optimizer_conv
    }
    mlflow.log_params(params)

    # Treinamento
    model.train()
    epochs = 1

    for epoch in range(epochs):
        print(f'------Época {epoch+1}------')
        for batch, (X, y) in enumerate(train_dataloader):
            X, y = X.to(device), y.to(device)

            pred = model(X)
            loss = criterion(pred, y)

            loss.backward()
            optimizer_conv.step()
            optimizer_conv.zero_grad()

            if batch % 100 == 0:
                print(f'Loss Atual: {loss.item()}')

        print('\nAvaliando o modelo...')
        model.eval()
        correct = 0
        total_loss = 0
        with torch.no_grad():
            for X_test, y_test in eval_dataloader:
                X_test, y_test = X_test.to(device), y_test.to(device)
                pred = model(X_test)
                total_loss += criterion(pred, y_test).item()
                predicted = (pred > 0.5).float()
                predicted = predicted.argmax(dim=1)
                correct += (predicted == y_test).sum().item()

        accuracy = correct / len(eval_dataloader.dataset)
        print(f'Acurácia: {accuracy * 100:.2f}%')

    # Testando modelo 

    print('\nTestando o modelo e logando métricas')
    model.eval()
    y_correct = []
    y_pred = []
    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            predicted = (pred > 0.5).float()
            predicted = predicted.argmax(dim=1)
            y_pred.append(predicted)
            y_correct.append(y)
    
    y_pred = torch.cat(y_pred, dim=0)
    y_correct = torch.cat(y_correct, dim=0)

    ## Métricas

    f1score = MulticlassF1Score(num_classes=3)
    precision_metric = MulticlassPrecision(num_classes=3)
    recall_metric = MulticlassRecall(num_classes=3)
    confusion_metric = MulticlassConfusionMatrix(num_classes=3)

    f1score.update(y_correct, y_pred)
    precision_metric.update(y_correct, y_pred)
    recall_metric.update(y_correct, y_pred)
    confusion_metric.update(y_correct, y_pred)

    f1sc = f1score.compute()
    precision = precision_metric.compute()
    recall = recall_metric.compute()
    confusion = confusion_metric.compute()

    # Logar métricas
    mlflow.log_metric("F1 Score", f1sc)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("Accuracy", accuracy)

    print('\nMétricas logadas')


    # Logando os pesos do modelo
    #model_path = 'resnet-50-finetuned.pth'
    #torch.save(model.state_dict(), model_path)  # Salva só os pesos
    #mlflow.log_artifact(model_path)  # Loga o arquivo

    # Logando o modelo

    mlflow.pytorch.log_model(model, "resnet-50-finetuned")


2025/04/28 16:51:02 WARNING mlflow.utils.autologging_utils: MLflow pytorch autologging is known to be compatible with 1.9.0 <= torch <= 2.6.0, but the installed version is 2.7.0+cu128. If you encounter errors during autologging, try upgrading / downgrading torch to a compatible version, or try upgrading MLflow.


------Época 1------
Loss Atual: 1.0864379405975342
Loss Atual: 0.6003836393356323

Avaliando o modelo...
Acurácia: 91.11%

Testando o modelo e logando métricas

Métricas logadas


2025/04/28 16:55:16 WARNING mlflow.utils.requirements_utils: Found torch version (2.7.0+cu128) contains a local version label (+cu128). MLflow logged a pip requirement for this package as 'torch==2.7.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/04/28 16:55:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [25]:
metric = MulticlassF1Score(num_classes=3)

metric.update(y_pred, y_correct)
teste = metric.compute()

In [26]:
teste

tensor(0.9467)

## Predict

In [ ]:

logged_model = r'file:///C:/Users/Lucas/Documents/GitHub/OrangeDetect/mlruns/862853957408423999/ef9efa9db0674007b7e41fab7eb272b1/artifacts/resnet-50-finetuned'
loaded_model = mlflow.pyfunc.load_model(logged_model)

img = cv2.imread(r'C:\Users\Lucas\Documents\GitHub\OrangeDetect\scripts\c (4610).jpg')
img = cv2.resize(img, (224, 224))

# Convert NumPy array to PIL image
img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor first
    transforms.Normalize([0.5], [0.5])  # Then normalize
])

img = transform(img)


# Add batch dimension
img = img.unsqueeze(0)

outputs = loaded_model.predict(img.cpu().numpy())  # Convert to numpy for compatibility with MLflow model

In [26]:
outputs

array([[ 2.9101045, -1.290107 , -1.6243376]], dtype=float32)

In [28]:
if outputs[0].argmax(axis=0) >= 0:
    print('Citrus')
elif outputs[0].argmax(axis=0) == 1:
    print('Healthy')
else:
    print('Melanose')

Citrus
